In [1]:
import numpy as np 
from pprint import pprint
from copy import deepcopy


from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
#from laboneq.contrib.example_helpers.generate_device_setup import generate_device_setup
from laboneq.dsl.device import DeviceSetup




descriptor = generate_descriptor(
    #pqsc=[""], # 장비 여러개 사용시
    shfqc_6=["DEV12256"],
    number_data_qubits=6,
    multiplex=True,
    number_multiplex=6,
    include_cr_lines=True,
    include_ef_lines=True,
    get_zsync=False,  # Only set to True when using real device
    save = True,
    filename="TG_RIP",
    ip_address="192.168.0.83"
)


#descriptor
setup = DeviceSetup.from_descriptor(yaml_text=descriptor, server_host="192.168.0.83")
#setup
setup.instruments[0].device_options = 'SHFQC/PLUS/QC6CH'
#setup.instruments

In [2]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=False, do_emulation=True)
#session.disconnect()

[2025.11.26 13:14:51.490] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/yalgaeahn/JSAHN/qubit-experiment/examples/laboneq_output/log
[2025.11.26 13:14:51.492] INFO    VERSION: laboneq 25.10.2
[2025.11.26 13:14:51.494] INFO    Connecting to data server at 192.168.0.83:8004
[2025.11.26 13:14:51.494] INFO    Connected to Zurich Instruments LabOne Data Server version 25.10.1.4 at 192.168.0.83:8004
[2025.11.26 13:14:51.496] INFO    Configuring the device setup
[2025.11.26 13:14:51.497] INFO    The device setup is configured


In [4]:
from qpu_types import TransmonQubit, TransmonQubitParameters
from helper import load_qubit_parameters, save_qubit_parameters
from operations import TransmonOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform



_loaded_qubits = load_qubit_parameters(filename="latest") 
#qubits = TransmonQubit.from_device_setup(setup) # Also can be done with load_qubit_parameters 
qubits = list(_loaded_qubits.values())





qpu = QPU(quantum_elements=qubits, quantum_operations=TransmonOperations())
#quantum_platform = QuantumPlatform(setup=setup, qpu=qpu)

# qpu.topology.add_edge("bus", "q16", "q17")
# qpu.topology.add_edge("bus", "q17", "q18")
# qpu.topology.add_edge("bus", "q18", "q19")
# qpu.topology.plot()




from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
#workflow.logbook.LoggingStore().activate()
#workflow.logbook.LogbookStore().deactivate()


In [7]:
qubits[0].parameters.resonance_frequency_ge =0

In [13]:
#workflow.show_fields(options)

AcquisitionType & AveragingMode


`INTEGRATION` & `CYCLIC`

STD 보려면  

`INTEGRATION` & `SINGLESHOT`

In [36]:
from experiments import modified_ramsey
from laboneq.simple import AcquisitionType, AveragingMode
q = qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temporary_parameters[q.uid] = temp_pars
#######################################################################
delays = np.linspace(0,10e-6,301)
detunings = 0.5e6,
pprint(q.readout_parameters())
#######################################################################
options = modified_ramsey.experiment_workflow.options()
options.update(False)
options.count(1024)
options.use_cal_traces(True)
#options.averaging_mode(AveragingMode.SINGLE_SHOT)
options.do_analysis(False)
# Build and run Ramsey workflow (no explicit command table options required)
ramsey_wf = modified_ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    options=options,
    temporary_parameters=temporary_parameters
)
ramsey_result = ramsey_wf.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
#print(ramsey_result.tasks['analysis_workflow'].output)

('measure',
 {'amplitude': 1.0,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2025.11.26 14:06:12.047] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.11.26 14:06:12.048] INFO      Workflow 'ramsey': execution started at 2025-11-26 05:06:12.046015Z           
[2025.11.26 14:06:12.048] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.11.26 14:06:12.066] INFO    Task 'temporary_qpu': started at 2025-11-26 05:06:12.066497Z
[2025.11.26 14:06:12.068] INFO    Task 'temporary_qpu': ended at 2025-11-26 05:06:12.068192Z
[2025.11.26 14:06:12.071] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-11-26 
[2025.11.26 14:06:12.071] INFO    05:06:12.070876Z
[2025.11.26 14:06:12.071] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-11-26 05:06:12.071760Z
[2025.11.26 14:06:12.073] INFO    Task

In [24]:
from experiments import ramsey
from laboneq.simple import AcquisitionType, AveragingMode
q = qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temporary_parameters[q.uid] = temp_pars
#######################################################################
delays = np.linspace(0,10e-6,301)
detunings = 0.5e6,
pprint(q.readout_parameters())
#######################################################################
options = ramsey.experiment_workflow.options()
options.update(False)
options.count(1024)
options.use_cal_traces(True)
options.averaging_mode(AveragingMode.SINGLE_SHOT)
options.do_analysis(False)
# Build and run Ramsey workflow (no explicit command table options required)
ramsey_wf = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    options=options,
    temporary_parameters=temporary_parameters
)
ramsey_result = ramsey_wf.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
#print(ramsey_result.tasks['analysis_workflow'].output)

('measure',
 {'amplitude': 1.0,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2025.11.26 13:34:09.594] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.11.26 13:34:09.595] INFO      Workflow 'ramsey': execution started at 2025-11-26 04:34:09.594740Z           
[2025.11.26 13:34:09.595] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.11.26 13:34:09.608] INFO    Task 'temporary_qpu': started at 2025-11-26 04:34:09.607780Z
[2025.11.26 13:34:09.608] INFO    Task 'temporary_qpu': ended at 2025-11-26 04:34:09.608830Z
[2025.11.26 13:34:09.612] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-11-26 
[2025.11.26 13:34:09.612] INFO    04:34:09.611928Z
[2025.11.26 13:34:09.612] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-11-26 04:34:09.612811Z
[2025.11.26 13:34:09.613] INFO    Task

In [30]:
from experiments import iq_time_trace as iq_t
options=iq_t.experiment_workflow.options()

In [35]:
options.count(2**13)
options

IQTrajExperimentWorkflowOptions(
│   do_analysis=True,
│   _task_options={
│   │   'create_experiment': IQTrajExperimentOptions(
│   │   │   acquisition_type=AcquisitionType.RAW,
│   │   │   averaging_mode=AveragingMode.CYCLIC,
│   │   │   count=8192,
│   │   │   repetition_mode=RepetitionMode.FASTEST,
│   │   │   repetition_time=None,
│   │   │   reset_oscillator_phase=False,
│   │   │   active_reset=False,
│   │   │   active_reset_repetitions=1,
│   │   │   active_reset_states='ge'
│   │   ),
│   │   'compile_experiment': CompileExperimentOptions(
│   │   │   compiler_settings=None
│   │   ),
│   │   'run_experiment': RunExperimentOptions(
│   │   │   include_results_metadata=False,
│   │   │   return_legacy_results=False
│   │   ),
│   │   'analysis_workflow': IQTrajAnalysisWorkflowOptions(
│   │   │   do_plotting=True,
│   │   │   do_plotting_iq_traj=True,
│   │   │   chunk_size=32,
│   │   │   _task_options={
│   │   │   │   'plot_iq_time_traces': BasePlottingOptions(
│   │   │   │   │   save_figures=True,
│   │   │   │   │   close_figures=True
│   │   │   │   )
│   │   │   }
│   │   )
│   }
)